In [1]:
import dotenv
import pendulum
from sqlalchemy import asc, cast
from sqlalchemy import create_engine, select, BigInteger
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, select, BigInteger
from config import Settings
from models import MessageSql

house_alias = "elm"
message_type = "layout.lite"
start_ms = pendulum.datetime(2025, 12, 13, 10, 30, tz='America/New_York').timestamp()*1000
end_ms = pendulum.datetime(2025, 12, 15, 17, 30, tz='America/New_York').timestamp()*1000

stmt = select(MessageSql).filter(
    MessageSql.message_type_name == message_type,
    MessageSql.from_alias == f"hw1.isone.me.versant.keene.{house_alias}.scada",
    MessageSql.message_persisted_ms <= cast(int(end_ms), BigInteger),
    MessageSql.message_persisted_ms >= cast(int(start_ms), BigInteger),
).order_by(asc(MessageSql.message_persisted_ms))

settings = Settings(_env_file=dotenv.find_dotenv())
engine = create_engine(settings.db_url_no_async.get_secret_value())
Session = sessionmaker(bind=engine)
session = Session()
result = session.execute(stmt)
messages = result.scalars().all()

print(f"Found {len(messages)} messages")

for m in messages:
    print(pendulum.from_timestamp(m.message_persisted_ms/1000, tz='America/New_York'))

# import matplotlib.pyplot as plt
# times = [pendulum.from_timestamp(m.message_persisted_ms/1000, tz='America/New_York') for m in messages]
# plt.scatter(times, [1]*len(messages))
# plt.show()


Found 18 messages
2025-12-13 13:19:18.964000-05:00
2025-12-13 14:55:44.551000-05:00
2025-12-13 15:05:08.606000-05:00
2025-12-13 15:31:33.481000-05:00
2025-12-13 16:46:58.611000-05:00
2025-12-13 17:10:19.958000-05:00
2025-12-13 17:14:14.294000-05:00
2025-12-13 18:55:49.437000-05:00
2025-12-14 02:07:40.528000-05:00
2025-12-14 02:35:19.981000-05:00
2025-12-14 02:43:36.885000-05:00
2025-12-14 02:52:14.563000-05:00
2025-12-14 03:00:05.369000-05:00
2025-12-14 03:33:37.566000-05:00
2025-12-14 14:08:07.617000-05:00
2025-12-14 16:37:34.766000-05:00
2025-12-14 20:55:06.491000-05:00
2025-12-15 08:07:31.047000-05:00


In [3]:
m = messages[0].payload
m['CriticalZoneList']

['living-kitchen', 'bedrooms', 'garage', 'master-bedrm']